<a href="https://colab.research.google.com/github/WorqHat/Langchain-Example/blob/main/Langchain_WorqhatAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install langchain_core
%pip install json

In [ ]:
import os
import json
import requests
from typing import Any, List, Mapping, Optional
from langchain_core.language_models.llms import LLM
from langchain_core.callbacks.manager import CallbackManagerForLLMRun

In [ ]:
class CustomLLM(LLM):
    """
    CustomLLM is a custom language model that interfaces with the Worqhat API to generate text based on a prompt.
    It supports additional parameters for fine-tuning the response, including training data for few-shot learning,
    control over the randomness of the response, and the ability to preserve conversation history.

    Attributes:
        n (int): An identifying parameter for instances of this class.
    """

    n: int

    @property
    def _llm_type(self) -> str:
        """Defines the type of the language model."""
        return "custom"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        """
        Generates a response to the given prompt using the Worqhat API.

        Parameters:
            prompt (str): The input prompt for the language model.
            stop (Optional[List[str]]): A list of stop sequences. Not permitted in this implementation.
            run_manager (Optional[CallbackManagerForLLMRun]): A manager for callbacks during the run. Not used.
            **kwargs (Any): Additional parameters supported by the API, such as training_data, randomness, etc.

        Returns:
            str: The generated response from the API.

        Raises:
            ValueError: If stop sequences are provided.
            HTTPError: If the API request fails.
        """
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")

        url = "https://api.worqhat.com/api/ai/content/v2"

        # Include additional parameters for API request
        payload = json.dumps({
            "question": prompt,
            "training_data": kwargs.get("training_data", "your name is johnny. when user asks whats your name. say johnny"),
            "preserve_history": kwargs.get("preserve_history", False),
            "stream_data": kwargs.get("stream_data", False),
            "history_object": kwargs.get("history_object", []),
            "randomness": kwargs.get("randomness", 0.5),
            "response_type": kwargs.get("response_type", "text"),  # The type of response to generate. Options: "text", "json"
            **kwargs
        })

        headers = {
            'Content-Type': 'application/json',
            'Accept': 'application/json',
            # 'Authorization': f'Bearer {os.getenv("WORQHAT_API_KEY")}'  # Load API key from environment variable
            'Authorization': f'Bearer sk-02e44d2ccb164c738a6c4a65dbf75e89'  # Load API key from environment variable
        }

        response = requests.post(url, headers=headers, data=payload)

        if response.status_code == 200:
            return response.text
        else:
            response.raise_for_status()

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """
        Returns identifying parameters for instances of this class.

        Returns:
            Mapping[str, Any]: A dictionary of identifying parameters.
        """
        return {"n": self.n}

In [ ]:
llm = CustomLLM(n=10)

In [ ]:
answer = llm.invoke("who created you?")

In [ ]:
response_dict = json.loads(answer)
try:
    # Attempt to parse the 'content' field as JSON
    parsed_content = json.loads(response_dict['content'])
    response_dict['content'] = parsed_content
except json.JSONDecodeError:
    # If 'content' is not a valid JSON, leave it as is
    pass

# Pretty print the JSON output
print(json.dumps(response_dict, indent=4))